In [1]:
import pandas as pd
import os
import requests, zipfile, io
import numpy as np


# Age

In [2]:
age = pd.read_csv("data/ipo_age.csv")
age

,CUSIP,Offer date,Company name,Ticker,Founding,PERM,VC dummy,Rollup,Dual,Post-issue shares,Internet
0,00846U101,19991117,Agilent Technologies Inc,A,1949,87432,0,0,0,.,0
1,7231103,19980925,"Admiralty Bancorp Inc, FL",AAABB,1987,86352,0,0,0,.,0
2,00790T100,20151111,Advanced Accelerator,AAAP,2002,15829,1,0,0,.,0
3,307108,20141002,AAC Holdings Inc,AAC,2009,14945,0,0,0,.,0
4,04543P100,20040204,Asset Acceptance Corp,AACC,1962,90020,0,0,0,.,0
...,...,...,...,...,...,...,...,...,...,...,...
12001,09060J20,20020624,Biodelivery Sciences Int'l Inc,NaN,1997,89445,1,0,0,.,0
12002,78460M209,20040817,"SNB Bancshares Inc,Texas",NaN,1985,90324,0,0,0,.,0
12003,33708M206,20041209,First State Financial Corp,NaN,-99,90480,0,0,0,.,0
12004,67071X100,20050620,Nuvim Inc,NaN,1999,.,0,0,0,.,0


In [3]:
age.columns = age.columns.str.replace(' ','_').str.lower()

In [4]:
age.drop(age.columns[np.r_[0,1,5,9]], axis=1, inplace=True)

In [5]:
age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12006 entries, 0 to 12005
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  12006 non-null  object
 1   ticker        11600 non-null  object
 2   founding      12006 non-null  object
 3   vc_dummy      12006 non-null  object
 4   rollup        12006 non-null  object
 5   dual          12006 non-null  object
 6   internet      12006 non-null  object
dtypes: object(7)
memory usage: 656.7+ KB


In [6]:
age = age[age['ticker'].notna()]
age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11600 entries, 0 to 11605
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  11600 non-null  object
 1   ticker        11600 non-null  object
 2   founding      11600 non-null  object
 3   vc_dummy      11600 non-null  object
 4   rollup        11600 non-null  object
 5   dual          11600 non-null  object
 6   internet      11600 non-null  object
dtypes: object(7)
memory usage: 725.0+ KB


**Manual Data Formatting**

In [7]:
age[age['ticker']=='BIGC'].replace('-99','2009',inplace=True)

/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [8]:
age[age['ticker']=='ANNX'].replace('-99','2011',inplace=True)

In [9]:
age.replace(['9','.'], '0',inplace=True)


In [10]:
messy_columns = age[(age["vc_dummy"].isin(['9','.']))
   | (age["dual"].isin(['9','.']))
   | (age["rollup"].isin(['.']))
   | (age["internet"].isin(['9','.']))]

messy_columns[messy_columns["rollup"].isin(['9','.'])] 

,company_name,ticker,founding,vc_dummy,rollup,dual,internet


<br>**vc_dummy** - is a dummy with 1 for VC and 2 for a subset of VC (growth capital)
<br>**dual** - dummy for multiple share class IPOs
<br>**rollup** - 0-1 dummy variable for whether the IPO was a rollup 
<br>**internet** - dummy with 1 if it is an internet-based  company 

In [11]:
age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11600 entries, 0 to 11605
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  11600 non-null  object
 1   ticker        11600 non-null  object
 2   founding      11600 non-null  object
 3   vc_dummy      11600 non-null  object
 4   rollup        11600 non-null  object
 5   dual          11600 non-null  object
 6   internet      11600 non-null  object
dtypes: object(7)
memory usage: 725.0+ KB


In [12]:
age.drop_duplicates(inplace=True)

<ipython-input-12-00299f27f858>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age.drop_duplicates(inplace=True)


In [13]:
age= age.groupby(['ticker']).max().reset_index()

In [14]:
age 

,ticker,company_name,founding,vc_dummy,rollup,dual,internet
0,A,Agilent Technologies Inc,1949,0,0,0,0
1,AAABB,"Admiralty Bancorp Inc, FL",1987,0,0,0,0
2,AAAP,Advanced Accelerator,2002,1,0,0,0
3,AAC,AAC Holdings Inc,2009,0,0,0,0
4,AACC,Asset Acceptance Corp,1962,0,0,0,0
...,...,...,...,...,...,...,...
10681,ZYMS,ZY MOS CORPORATION,1978,0,0,0,0
10682,ZYNE,Zynerba Pharmaceuticals Inc,2007,1,0,0,0
10683,ZYNX,Zynaxis,1987,1,0,0,0
10684,ZYTC,Zytec Corp,1983,0,0,0,0


# Save Collected Data 

In [15]:
import joblib 
joblib.dump(age,"age.pkl")

['age.pkl']

In [16]:
ls


1_collect_clean_AGE.ipynb       data/
2_collect_clean_MANAGERS.ipynb  ipo_180.pkl
3a_collect_SEC.ipynb            managers_all.pkl
3b_clean_SEC.ipynb              managers_melt.pkl
4a_collect_YF1.ipynb            sec_10k.pkl
4b_clean_YF1.ipynb              sec_10k_raw.pkl
5a_collect_YF2.ipynb            yf1.pkl
5b_clean_YF2.ipynb              yf1_raw.pkl
age.pkl                         yf2.pkl
crowdfunding.pkl
